# Model Learning


To Do: 

Andere Modelle implementieren,

Evalutaion erstellen der verschiedenen Modelle und vergleichen,

Schauen ob schon erste Testläufe mit den bereits preprocesseden daten möglich sind, 

Pipeline

Hyper Parameter Optimization

In [6]:
import pandas as pd


## Import Data 
And getting familiar with data set

In [7]:
#Import Data
df = pd.read_csv('../data/processed/export_v1.csv') #not fully preprocessed yet

/var/folders/gt/wnv6zp0s7w92jy1_p2n6shsr0000gn/T/ipykernel_22388/849369748.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/processed/export_v1.csv') #not fully preprocessed yet


In [8]:
df.head()

,resultId,raceId,driverId,constructorId,number,grid,year,round,circuitId,race_name,...,avg_pit_stop_lap,std_pit_stop_lap,avg_pit_stop_driver,avg_pit_stop_team,avg_pit_stop_track,avg_pit_stop_season,rolling_avg_pit_stop_driver,rolling_avg_pit_stop_team,rolling_avg_pit_stop_track,rolling_avg_pit_stop_season
0,1,18,1,1,22,1,2008,1,1,Australian Grand Prix,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,18,2,2,3,5,2008,1,1,Australian Grand Prix,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,18,3,3,7,7,2008,1,1,Australian Grand Prix,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,18,4,4,5,11,2008,1,1,Australian Grand Prix,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,18,5,1,23,3,2008,1,1,Australian Grand Prix,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26699 entries, 0 to 26698
Data columns (total 43 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   resultId                     26699 non-null  int64  
 1   raceId                       26699 non-null  int64  
 2   driverId                     26699 non-null  int64  
 3   constructorId                26699 non-null  int64  
 4   number                       26699 non-null  object 
 5   grid                         26699 non-null  int64  
 6   year                         26699 non-null  int64  
 7   round                        26699 non-null  int64  
 8   circuitId                    26699 non-null  int64  
 9   race_name                    26699 non-null  object 
 10  race_date                    26699 non-null  object 
 11  race_time                    26699 non-null  object 
 12  fp1_date                     26699 non-null  object 
 13  fp2_date        

## Additional Preprocessing

In which shape is the preprocessed data? Are additional preprocessing techniques necessary for different models ?

Is there already an train-test(-validation) split?

Binning, Boosting & stratified sampling

Encoding necessary?

In [ ]:
df_target = df['position'] #target variable not yet in dataframe

In [ ]:
#Train Test Split (mit stratified sampling)

from sklearn.model_selection import train_test_split

df_train, df_test, df_target_train, df_target_test = train_test_split(
    df, df_target,test_size=0.2, random_state=42, stratify=df_target)

print("=======TRAIN=========")
display(df_train)
display(df_target_train)

In [ ]:
df_test_wo_target = df_test.drop(columns=['position'])
df_train_wo_target = df_train.drop(columns=['position'])

## Random Forrest

Hyperparameters that can be optimized are  max_depth, n_estimators, min_samples_leaf, min_samples_split, max_features, criterion: {“gini”, “entropy”}.

Random_state is only used for the reproducibility of results.


In [ ]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(max_depth=2, random_state=0) #Modell initialisieren

random_forest.fit(df_train, df_target_train) #Lernvorgang des Modells

In [ ]:
df_rf_prediction = random_forest.predict(df_test_wo_target) #Vorhersage des Modells

### Hyperparameter Optimization

In [ ]:
#Hyperparameter tuning trough grid search

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Definiere das Modell
clf = RandomForestClassifier(random_state=42)

# Definiere den Parameter-Raster
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Führe die Grid-Suche durch
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(df_train_wo_target, df_target_train)

# Beste Parameter anzeigen
print("Beste Parameter:", grid_search.best_params_)

## Naive Bayes
Assumes different attributes are independent of each other. (Did we remove duplicates/ very similar attributes?)

[Naive Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html) is implemented in different variations in scikit-learn.
They differ mainly by the assumptions they make regarding the distribution of $P(x_i|y)$


- [```GaussianNB``` class](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html#sklearn.naive_bayes.GaussianNB) implements the Naive Bayes classifier for continious (numeric) features. Likelihood of the features is assumed to be Gaussian
- [```MultinomialNB``` class](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html) implements the Naive Bayes classifier for discrete (categorical) features (multinomially distributed data)
- [```BernoulliNB``` class](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.BernoulliNB.html) assumes multivariate Bernoulli distributions
- [```CategoricalNB``` class](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.CategoricalNB.html) assumes that each feature has its own categorical distribution

Could the assumption about the distributions of $P(x_i|y)$ be regarded as an hyperparameter wich can be optimized? 

Only 4 Options, so try what works best.

In [ ]:
from sklearn.naive_bayes import GaussianNB
naive_bayes = GaussianNB()
naive_bayes.fit(df_train, df_target_train)

In [ ]:
df_nb_prediction = naive_bayes.predict(df_test_wo_target)

## Evaluation of Models

(Eventuell seperates Dokument)

### Confusion Matrix for Random Forrest

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

display(df_target_test)
display(df_rf_prediction) #still need to predict

print("Confusion Matrix:")
print(confusion_matrix(df_target_test, df_rf_prediction))
print()

print("Accuracy: {}".format(accuracy_score(df_target_test, df_rf_prediction)))

### Classificaion Report (Recall and Precision)

In [ ]:
from sklearn.metrics import classification_report
print("Classification Report:")
print(classification_report(df_target_test, df_rf_prediction, target_names=label_encoder.classes_))

### Cross Validation

In [ ]:
#Implement a cross validation split to get a better estimate about performance of used models.
#Example for Random Forrest

### Visual Evaluation (ROC Curves)

Classifiers necessary which deliver confidence scores of their predicitons. It has to be possible to apply the  ```predict_proba()``` function.

## Ensembles

Let the different models vote for final prediction.

Possabilities: 
Stacking,
Voting

### Stacking

Stacking example for the Random Forrest and Naive Bayes Classifier with a Decision Tree classifier as the meta classifier.

In [ ]:
#Stacking Example for Naive Bayes and Random Forrest

from sklearn.tree import DecisionTreeClassifier #Meta Learner
from sklearn.preprocessing import LabelEncoder

# Stacking: scikit-learn has no implementation for Stacking. You can use this -> https://rasbt.gith2ub.io/mlxtend/
# HINT: mlxtend can only work with numerical labels. You can use the LabelEncoder to transform your labels.

from mlxtend.classifier import StackingClassifier

estimators = {'RandomForrest': RandomForestClassifier(max_depth=2, random_state=0), 'NaiveBayes': GaussianNB()} #Possible to extend to even more classifiers

encoder = LabelEncoder()
df_target_train_encoded = encoder.fit_transform(df_target_train)
df_target_test_encoded = encoder.transform(df_target_test)

ensemble = StackingClassifier(
    classifiers=list(estimators.values()),
    meta_classifier=DecisionTreeClassifier()
)
estimators['Stacking'] = ensemble

for e_name, e in estimators.items():
    evaluate_classifier(e_name, e, df_train_wo_target, df_target_train_encoded, df_test_wo_target, df_target_test)

#Does it improve accuracy?